# Задание 2. Расчет процентной ставки

### 1. Считаем переданные в JSON-объекте параметры
Если бы задание было более масштабным, можно было бы подготовить подбробную документацию с описанием API, ресурсов и формата запросов, интрукцией пользователя. А пока просто приймем что для работы с нашим API требуется, чтобы запрос содержал параметры в указаном порядке: 

loan - сумма ссуды  в у.е.,  float()

rate-годовая процентная ставка в процентах, float()

period-кредитный период, целое неотрицательное число, кратное количеству месяцев, на которое выдается кредит,  int().

Если бы можно использовать Python библиотеки, то я реализовала бы код следующими образом:

In [1]:
import json as js

# читаем json-данные из файла и преобразуем в словарь
with open('data.json', encoding = 'UTF-8') as file:
    data = js.load(file)

"""Здесь можно сделать проверку безопасности: какие введены данные в каждой переменной, соотвествуют ли значения требуемому 
формату, проверки на дурака, переполнение буфера и др."""
    
print('Сумма кредита составляет = ', data["loan"], "у.е.")
print('Годовая процентная ставка равна = ', data["rate"], "%")
print('Период кредитования:',data["period"] , "месяцев")

Сумма кредита составляет =  215000 у.е.
Годовая процентная ставка равна =  7.1 %
Период кредитования: 36 месяцев


Посольку по условиям задания использовать какие-либо библиотеки запрещено, а также чтоб продемонстрировать свои зания я реализую парсинг JSON-строки вручную, используя встроенные методы работы со строками Python. Пусть мы получили запрос, где переметры расчета должны быть заданы в следующем виде и порядке: {"loan":215000, "rate":7.1, "period":12}

In [2]:
JSON_string = '{"loan":100000, "rate":10, "period":12}'  #Внесем значение JSON-строки в переменную JSON_string:

# читаем json-данные из строки в переменные:
loan=float(JSON_string[JSON_string.index("loan")+6:JSON_string.index("rate")-3])
rate=float(JSON_string[JSON_string.index("rate")+6:JSON_string.index("period")-3])
period=int(JSON_string[JSON_string.index("period")+8:JSON_string.index("}")])

print('Сумма кредита составляет = ', loan, "у.е.")
print('Годовая процентная ставка равна = ', rate, "%")
print('Период кредитования:', period , "месяцев")

Сумма кредита составляет =  100000.0 у.е.
Годовая процентная ставка равна =  10.0 %
Период кредитования: 12 месяцев


### 2. Рассчитаем процентную ставку и выведем результат
В случае ежемесячных платежей сумма процентов рассчитывается по формуле:

СП = СКост. x ПС   (1),

где СКост. - остаток задолженности по кредиту, на который начисляются проценты;

ПС - месячная процентная ставка (1/12 годовой процентной ставки, деленная на 100)


http://www.consultant.ru/edu/student/consultation/protsenty_po_kreditu/
© КонсультантПлюс, 1992-2021

В условии задачи было указано, что выплата осуществляется единоразово в конце периода. Приймем также что проценты начисляются тоже единоразово на конец периода, а период кратен месяцу.  Тогда для вычисления выплачиваемых на конец период процентов воспользуемся формулой:

СП = сумма кредита x ПС x период кредитования в  месяцах  (2).

In [3]:
total_interest = loan*rate*0.01/12*period    #вычисляется "переплата" - проценты требуемые к выплате по кредиту на конец периода
print('Начисленные за %i месяцев проценты по кредиту =' % period, total_interest, "у.е.")


total_paid = loan+total_interest          #вычисляется сумма к выплате на конец периода: сумма кредита + начисленные проценты
print('Всего через %i месяцев должна быть выплачена сумма =' % period,  total_paid, "у.е.")

Начисленные за 12 месяцев проценты по кредиту = 10000.0 у.е.
Всего через 12 месяцев должна быть выплачена сумма = 110000.0 у.е.


### 3. Немного кода сверх задачи. Для удовольствия 
Если в разрез с условиями задания принять, что платежи по погашению кредита будут совершаться ежемесячно равными долями (дифференцированный платеж), то выплаченная сумма будет меньше, поскольку тогда сумма основного долга, на которую рассчитываются проценты, будет постоянно уменьшаться. 

В этом случае сумму выплаченных прцентов по кредиту можно вычислить следующим образом:

In [4]:
"""Результат вычислений процентов за каждый месяц и ежемесячного платежа будем заносить в массивы, чтобы иметь к ним доступ 
позже. Результат всех делений будем округлять до 2х знаков после запятой, т.к. мы не можем оперировать суммами меньше копейки """

def monthly_interest_and_payments (loan, rate, period):
    principal= round(loan/period, 2)      #размер ежемесячного платежа в пользу основного долга
    balance=loan                          #остаток основного долга
    payments = []                         #список ежемесячных платежей
    interest = []                         #список начисленных процентов за каждый месяц
    
    #будем вычислять размер процентов в каждом месяце, уменьшая сумму основного долга, и ежемесячный платеж
    for i in range(0, period):            
        interest_i=round(balance*rate*0.01/12, 2)    #проценты, начисленные в очередном месяце i (формула (1))
        interest.append(interest_i)
        balance=round(balance-principal, 2)          #уменьшение суммы основного долга в i-тый раз
        """поскольку ежемесячный платеж по телу долга paid_loan  мы округли до двух сотых, то после последнего вычитания его из 
        тела долга может остаться остаток, учтем его в последнем платеже:"""
        if i== period-1:                                  
                principal=principal+round(balance,2)                   
        payments.append(interest_i+principal)        #размер платежа за месяц i
    return interest, payments

interest, payments = monthly_interest_and_payments(loan, rate, period)
print('Начисленные за %i месяцев проценты по кредиту =' % period, sum(interest), "у.е.")
print('Всего за %i месяцев должно быть выплачено =' % period, sum(payments), "у.е.")

Начисленные за 12 месяцев проценты по кредиту = 5416.66 у.е.
Всего за 12 месяцев должно быть выплачено = 105416.66 у.е.


Выведем все полученные данные по кредиту:

In [7]:
import numpy as np
import pandas as pd

total=pd.DataFrame(np.array([list(range(1,period+1)), interest, payments]).T, columns = ['Месяц', 'Процент по кредиту', 'Ежемесячный платеж']).set_index('Месяц')
print (total)


       Процент по кредиту  Ежемесячный платеж
Месяц                                        
1.0                833.33             9166.66
2.0                763.89             9097.22
3.0                694.44             9027.77
4.0                625.00             8958.33
5.0                555.56             8888.89
6.0                486.11             8819.44
7.0                416.67             8750.00
8.0                347.22             8680.55
9.0                277.78             8611.11
10.0               208.33             8541.66
11.0               138.89             8472.22
12.0                69.44             8402.81
